# word2vec IMDB data

Training word2vec embeddings on the IMDB database and experimenting.

Referência: Tutorial Kagggle ["Bag of Words meets Bags of Popcorn"](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-2-word-vectors)

In [13]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
import pandas as pd
import gensim

In [7]:
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [88]:
train

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
5,"""8196_8""",1,"""I dont know why people think this is such a b..."
6,"""7166_2""",0,"""This movie could have been very good, but com..."
7,"""10633_1""",0,"""I watched this video at a friend's house. I'm..."
8,"""319_1""",0,"""A friend of mine bought this film for £1, and..."
9,"""8713_10""",1,"""<br /><br />This movie is full of references...."


In [8]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [9]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))
/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/emannuelcarvalho/.virtual

Parsing sentences from unlabeled set


/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a U

In [ ]:
y = train["sentiment"]
X = []
for review in train["review"]:
    sentences = review_to_sentences(review, tokenizer)
    words = []
    for sentence in sentences:
        words += sentence
    X.append(sentence)

In [123]:
print len(train["sentiment"]), len(train["review"]), len(X), X[0]
# print review_to_sentences(train[0], tokenizer)

25000 25000 266551 [u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']


In [14]:
model = gensim.models.Word2Vec(sentences, min_count=1)

In [100]:
def vectorize(sentence):
    return [model[word] for word in sentence]

In [127]:
review_to_sentences("this is my name. And this is yours", tokenizer)

[[u'this', u'is', u'my', u'name'], [u'and', u'this', u'is', u'yours']]

In [109]:
X = [vectorize(sentence) for sentence in X]

TypeError: unhashable type: 'list'

In [107]:
len(X)

34

In [67]:
print(model.wv.most_similar(positive=['he', 'her'], negative=['she']))

[(u'his', 0.9265413880348206), (u'him', 0.7602269649505615), (u'himself', 0.6485036015510559), (u'herself', 0.5875888466835022), (u'reckless', 0.5832635164260864), (u'their', 0.555915117263794), (u'cancer', 0.5551137924194336), (u'jed', 0.5519363880157471), (u'nanon', 0.5372742414474487), (u'gauri', 0.5353655815124512)]


In [29]:
acc = model.accuracy('questions-words.txt')

In [36]:
[(d.keys()[1], d[d.keys()[1]]) for d in acc]

[('section', u'capital-common-countries'),
 ('section', u'capital-world'),
 ('section', u'currency'),
 ('section', u'city-in-state'),
 ('section', u'family'),
 ('section', u'gram1-adjective-to-adverb'),
 ('section', u'gram2-opposite'),
 ('section', u'gram3-comparative'),
 ('section', u'gram4-superlative'),
 ('section', u'gram5-present-participle'),
 ('section', u'gram6-nationality-adjective'),
 ('section', u'gram7-past-tense'),
 ('section', u'gram8-plural'),
 ('section', u'gram9-plural-verbs'),
 ('section', 'total')]

In [54]:
for i in range(0, len(acc)):
    print(acc[i][acc[i].keys()[1]], len(acc[i]['correct']), len(acc[i]['incorrect']))#, len(acc[i]['correct']/len(acc[i]['incorrect']))

(u'capital-common-countries', 26, 246)
(u'capital-world', 19, 269)
(u'currency', 0, 40)
(u'city-in-state', 6, 851)
(u'family', 241, 179)
(u'gram1-adjective-to-adverb', 47, 883)
(u'gram2-opposite', 42, 608)
(u'gram3-comparative', 643, 689)
(u'gram4-superlative', 217, 539)
(u'gram5-present-participle', 271, 541)
(u'gram6-nationality-adjective', 80, 1149)
(u'gram7-past-tense', 382, 950)
(u'gram8-plural', 273, 657)
(u'gram9-plural-verbs', 352, 350)
('total', 2599, 7951)


In [82]:
model.wv.syn0.shape

(123504, 100)

## Keras model

In [69]:
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.datasets import imdb
from keras import backend as K

from theano import function

Using Theano backend.


In [85]:
max_features = 20000
max_len = 500
weights = model.wv.syn0
lstm_model = Sequential()
# lstm_model.add(Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights]))
# lstm_model.add(Embedding(max_features, 128, input_length = max_len))
lstm_model.add(LSTM(100))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))

print(lstm_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 100)         12350400  
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 12,430,901
Trainable params: 12,430,901
Non-trainable params: 0
_________________________________________________________________
None


In [84]:
max_features = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words = max_features)

In [86]:
lstm_model.compile(loss='binary_crossentropy',
              optimizer = 'adam',
              metrics=["accuracy"])
print("Train..")
batch_size = 30
score = lstm_model.fit(X_train, y_train, batch_size = batch_size,
                      nb_epoch = 4, validation_data = (X_test, y_test))

Train..
Train on 25000 samples, validate on 25000 samples
Epoch 1/4


ValueError: Bad input argument to theano function with name "/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/keras/backend/theano_backend.py:1154" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):
  File "/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-85-aa344cb3472f>", line 5, in <module>
    lstm_model.add(Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights]))
  File "/Users/emannuelcarvalho/.virtualenvs/primogenithon/lib/python2.7/site-packages/keras/models.py", line 429, in add
    dtype=layer.dtype, name=layer.name + '_input')
  File "/Users/emannuelcarvalh